# Preparation For Topic Modelling

Eventually to include tf-idf threshold

In [1]:
import os
import pandas as pd
from nltk import word_tokenize
import string

In [2]:
sdf = pd.read_csv('transcripts/speeches.csv')
tdf = pd.read_csv('transcripts/transcripts.csv', index_col=0)

# Quick fix, to be removed after the next full extract from the transcripts
tdf.loc[36151, 'content'] = ''

A function to eliminate utterances less than 5 words long. Not currently sure if I will use this. I would if I was replicating Shapiro Wilson 2021, but that is no longer the plan. Keeping it around for now.

In [1]:
def long_enough(text):
    cleaned_tokens = [token for token in word_tokenize(str(text)) if token not in string.punctuation]
    if len(cleaned_tokens) < 5:
        return False
    return True

In [4]:
tdf_filtered = tdf[tdf.content.apply(long_enough) == True]

In [5]:
tdf_filtered = tdf_filtered.reset_index().drop('level_0', axis=1)

Another idea from Shapiro and Wilson, which I don't know if I will use. The idea is to remove "stand alone utterances" where the speakers of the utterances before and after is different. In practice, I don't know how useful this is. It is possible I am misinterpreting their meaning. Either way, for now I am not using this but I'm leaving it around for now.

In [6]:
def check_bef_aft_speakers(row):
    row_ind = row.name
    # print(row_ind)
    if row_ind == 0 or row_ind == len(tdf) - 1:
        return True
    if tdf.loc[row_ind].date != tdf.loc[row_ind + 1].date:
        return True
    prev_speaker = tdf.loc[row_ind - 1].speaker
    next_speaker = tdf.loc[row_ind + 1].speaker
    curr_speaker = tdf.loc[row_ind].speaker

    print('---------')
    print(row.content)
    print(prev_speaker)
    print(curr_speaker)
    print(next_speaker)

    if curr_speaker != prev_speaker and curr_speaker != next_speaker and prev_speaker != next_speaker:
        print('get rid')
        return False
    return True

In [7]:
# tdf_filtered['test'] = tdf_filtered.apply(check_bef_aft_speakers, axis=1)

In [8]:
tdf_filtered

,speaker,content,index,date,type
0,CHAIR YELLEN.,"Good morning, everybody. As you know, we had ...",0,2015-03-18,meeting
1,CHAIR YELLEN.,Thank you. Without objection. I am going to ...,2,2015-03-18,meeting
2,MR. POTTER.,"Thank you, Madam Chair. We will be splitting t...",3,2015-03-18,meeting
3,MR. KOCHERLAKOTA.,"Yes. Actually, it is more of a comment than a ...",5,2015-03-18,meeting
4,MR. POTTER.,"In the past week, that is true. The TIPS break...",6,2015-03-18,meeting
...,...,...,...,...,...
37005,CHAIRMAN BERNANKE.,"Thank you very much. Thank you, all, for bear...",373,2012-03-13,meeting
37006,MR. LACKER.,Earlier in the meeting about today's statement...,374,2012-03-13,meeting
37007,CHAIRMAN BERNANKE.,"Well, I asked for discussion of that point tod...",375,2012-03-13,meeting
37008,CHAIRMAN BERNANKE.,And I was very clear that that is certainly no...,377,2012-03-13,meeting


## Bi and Trigram enhancements

In [3]:
bigrams = set()

with open(os.path.join('ngrams', 'bigrams')) as bg_file:
    for bg in bg_file.readlines():
        bigrams.add(bg.strip())

In [4]:
trigrams = set()

with open(os.path.join('ngrams', 'trigrams')) as tg_file:
    for tf in tg_file.readlines():
        trigrams.add(tf.strip())

In [5]:
def ngram_enhance(raw_string):
    to_add = []
    raw_string = str(raw_string)
    for bg in bigrams:
        if str(bg) in raw_string.lower():
            to_add.append('_'.join(bg.split(' ')))
    for tg in trigrams:
        if str(tg) in raw_string.lower():
            to_add.append('_'.join(tg.split(' ')))

    return raw_string + ' ' + ' '.join(to_add)

In [7]:
sdf['text_enhanced'] = sdf['text'].apply(ngram_enhance)
tdf['text_enhanced'] = tdf['content'].apply(ngram_enhance)

In [ ]:
tdf.to_csv('transcripts_enhanced.csv')

In [ ]:
sdf.to_csv('speeches_enhanced.csv')

## Create Speaker-Meeting observations

In [ ]:
tdf_e = pd.read_csv('transcripts_enhanced.csv')

In [3]:
speaker_grouped = tdf.groupby(['date', 'speaker'])['content'].agg(' '.join).reset_index()

In [4]:
speaker_grouped.to_csv('speaker_grouped.csv')